In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
rootPath = "./processed/"
ioPath = "./processed_io_data/"

print("File paths configured:")
print(f"  Processed data: {rootPath}")
print(f"  I-O data: {ioPath}")

File paths configured:
  Processed data: ./processed/
  I-O data: ./processed_io_data/


# **Creating Master Dataset**

## **Loading all Datasets**

In [3]:
# 1. Commodity Prices (CMO)
commodity_prices = pd.read_csv(rootPath + 'proc_cmo_monthly.csv')
# Drop the unnamed index column
commodity_prices = commodity_prices.drop(columns=['Unnamed: 0'], errors='ignore')
if 'Date' in commodity_prices.columns:
    commodity_prices = commodity_prices.rename(columns={'Date': 'date'})
commodity_prices['date'] = pd.to_datetime(commodity_prices['date'])
print(f"  ✓ Commodity prices: {commodity_prices.shape}")
display(commodity_prices.head(2))

  ✓ Commodity prices: (180, 32)


,5,date,CRUDE_PETRO,WHEAT_US_HRW,RICE_05,COPPER,ALUMINUM,CRUDE_PETRO_logret,WHEAT_US_HRW_logret,RICE_05_logret,...,COPPER_vol_6m,COPPER_vol_12m,ALUMINUM_vol_3m,ALUMINUM_vol_6m,ALUMINUM_vol_12m,CRUDE_PETRO_shock,WHEAT_US_HRW_shock,RICE_05_shock,COPPER_shock,ALUMINUM_shock
0,606,2010-01-01,77.121088,201.190932,568.8,7386.250,2235.150,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1,607,2010-02-01,74.763016,194.000000,535.0,6848.175,2048.925,-0.031053,-0.036396,-0.061262,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0


In [4]:
# 2. Climate Data (ONI)
climate_oni = pd.read_csv(rootPath + 'climate_oni_clean.csv')
climate_oni = climate_oni.drop(columns=['Unnamed: 0'], errors='ignore')
# Date column already exists
climate_oni['Date'] = pd.to_datetime(climate_oni['Date'])
climate_oni = climate_oni.rename(columns={'Date': 'date'})
print(f"\n  ✓ Climate ONI: {climate_oni.shape}")
display(climate_oni.head(2))


  ✓ Climate ONI: (96, 9)


,Year,Month,ONI,Month_num,date,ENSO_Phase,ONI_lag_1m,ONI_lag_3m,ONI_lag_6m
0,2010,DJF,1.3,1,2010-01-01,El Nino,NaN,NaN,NaN
1,2010,JFM,1.2,1,2010-01-01,El Nino,1.3,NaN,NaN


In [5]:
# 3. IIP Sectoral
iip_sectoral = pd.read_csv(rootPath + 'iip_sectoral.csv')
iip_sectoral['date'] = pd.to_datetime(iip_sectoral['date'])
print(f"\n  ✓ IIP Sectoral: {iip_sectoral.shape}")
display(iip_sectoral.head(2))


  ✓ IIP Sectoral: (3519, 6)


,sector_name,date,iip_index,iip_mom_growth,iip_yoy_growth,is_energy_intensive
0,General Index,2012-04-01,99.3,NaN,NaN,False
1,General Index,2012-05-01,105.0,5.740181,NaN,False


In [6]:
# 4. WPI Inflation
wpi_inflation = pd.read_csv(rootPath + 'wpi_inflation.csv')
wpi_inflation['date'] = pd.to_datetime(wpi_inflation['date'])
print(f"\n  ✓ WPI Inflation: {wpi_inflation.shape}")
display(wpi_inflation.head(2))


  ✓ WPI Inflation: (495, 4)


,category,date,wpi_index,wpi_inflation
0,(A). FOOD ARTICLES,2011-04-01,100.0,NaN
1,(A). FOOD ARTICLES,2011-05-01,100.0,NaN


In [7]:
# 5. GDP Quarterly
gdp_quarterly = pd.read_csv(rootPath + 'gdp_quarterly.csv')
gdp_quarterly['date'] = pd.to_datetime(gdp_quarterly['date'])
print(f"\n  ✓ GDP Quarterly: {gdp_quarterly.shape}")
display(gdp_quarterly.head(2))


  ✓ GDP Quarterly: (157, 4)


,date,gdp_constant,gdp_current,gdp_growth_yoy
0,2011-04-01,2.102863e+06,2.043496e+06,NaN
1,2011-05-01,2.102863e+06,2.043496e+06,NaN


In [8]:
# 6. Global Macro (OECD)
global_macro = pd.read_csv(rootPath + 'global_macro.csv')
global_macro['date'] = pd.to_datetime(global_macro['date'])
print(f"\n  ✓ Global Macro: {global_macro.shape}")
display(global_macro.head(2))


  ✓ Global Macro: (4, 20)


,date,ARG,AUS,BRA,CAN,CHN,DEU,EA20,EU27_2020,FRA,GBR,IDN,IND,ITA,KOR,SAU,TUR,USA,ZAF,g20_avg_cpi_growth
0,2024-09-01,209.0011,NaN,4.424788,1.640379,0.4,1.8,1.7,2.1,1.4,1.7,1.84,4.218182,0.7,1.595038,1.695224,49.5,2.440633,NaN,17.884709
1,2024-10-01,192.9950,NaN,4.758112,2.017654,0.3,2.4,2.0,2.3,1.6,2.3,1.71,4.407515,1.0,1.253642,1.884216,48.7,2.597905,2.774923,16.176410


In [9]:
# 7. Trade Data
trade_bilateral = pd.read_csv(rootPath + 'trade_india_bilateral.csv')
# Parse date from YYYY-MXX format
if 'date' not in trade_bilateral.columns:
    # Date appears to be in a column - find it
    date_col = [col for col in trade_bilateral.columns if 'M' in str(trade_bilateral[col].iloc[0])]
    if date_col:
        trade_bilateral['date'] = pd.to_datetime(trade_bilateral[date_col[0]], format='%Y-M%m')
    else:
        print("  ⚠️ Warning: Could not find date column in trade data")
else:
    trade_bilateral['date'] = pd.to_datetime(trade_bilateral['date'])
print(f"\n  ✓ Trade Bilateral: {trade_bilateral.shape}")
display(trade_bilateral.head(2))


  ✓ Trade Bilateral: (9720, 6)


,COUNTRY,COUNTERPART_COUNTRY,TRADE_FLOW,commodity_group,date,trade_value_usd
0,India,Qatar,Trade balance goods,Other,2010-01-01 00:00:01,NaN
1,India,Qatar,Exports of goods,Other,2010-01-01 00:00:01,22.638913


In [10]:
# 8. Network Metrics (I-O)
network_metrics = pd.read_csv(ioPath + 'network_metrics.csv')
network_metrics = network_metrics.drop(columns=['Unnamed: 0'], errors='ignore')
print(f"\n  ✓ Network Metrics: {network_metrics.shape}")
display(network_metrics.head(2))

print("\n" + "="*70)
print("All datasets loaded successfully!")
print("="*70)


  ✓ Network Metrics: (131, 10)


,sector_id,sector_name,backward_linkage,forward_linkage,is_key_sector,degree_centrality,betweenness_centrality,closeness_centrality,eigenvector_centrality,pagerank
0,1,Paddy,2.056934,1.990450,False,0.238462,0.002546,0.449332,0.043067,0.005365
1,2,Wheat,1.507685,1.354732,False,0.269231,0.002786,0.447604,0.037984,0.004517



All datasets loaded successfully!


# Sector Mapping [From IIP to IO]

In [11]:
# Comprehensive mapping dictionary
sector_mapping = {
    # Food & Beverages
    'Manufacture of food products': 'Miscellaneous food\n products',  # Sector 43
    'Manufacture of beverages': 'Beverages',  # Sector 45
    
    # Textiles & Apparel
    'Manufacture of textiles': 'Cotton textiles',  # Sector 48 (aggregate for all textiles)
    'Manufacture of wearing apparel': 'Ready made garments',  # Sector 55
    'Manufacture of leather and related products': 'Leather and leather\n products',  # Sector 61
    
    # Wood, Paper, Printing
    'Manufacture of paper and paper products': 'Paper, Paper products and\n newsprint',  # Sector 57
    'Printing and reproduction of recorded media': 'Publishing, printing and\n allied activities',  # Sector 58
    
    # Chemicals & Petroleum
    'Manufacture of chemicals and chemical products': 'Other chemicals',  # Sector 73
    'Manufacture of pharmaceuticals, medicinal chemical and botanical products': 'Drugs and medicine',  # Sector 74
    'Manufacture of coke and refined petroleum products': 'Petroleum products',  # Sector 64
    'Manufacture of rubber and plastics products': 'Plastic products',  # Sector 63
    
    # Non-metallic minerals
    'Manufacture of other non-metallic mineral products': 'Cement',  # Sector 76
    
    # Metals
    'Manufacture of basic metals': 'Iron and steel foundries',  # Sector 80
    'Manufacture of fabricated metal products, except machinery and equipment': 'Miscellaneous metal\n products',  # Sector 83
    
    # Machinery & Equipment
    'Manufacture of machinery and equipment n.e.c.': 'Other non-electrical\n machinery',  # Sector 88
    'Manufacture of computer, electronic and optical products': 'Electronic\n equipments(incl.TV)',  # Sector 94
    'Manufacture of electrical equipment': 'Electrical industrial\n Machinery',  # Sector 89
    
    # Transport Equipment
    'Manufacture of motor vehicles, trailers and semi-trailers': 'Motor vehicles',  # Sector 99
    'Manufacture of other transport equipment': 'Other transport\n equipments',  # Sector 102
    
    # Other Manufacturing
    'Manufacture of furniture': 'Furniture & Fixtures',  # Sector 59
    'Manufacture of tobacco products': 'Tobacco Products',  # Sector 46
    'Other manufacturing': 'Miscellaneous\n manufacturing',  # Sector 106
}

In [12]:
# Apply mapping to IIP data
iip_sectoral['io_sector_name'] = iip_sectoral['sector_name'].map(sector_mapping)

# For unmapped sectors (like "General Index"), keep original name
iip_sectoral['io_sector_name'] = iip_sectoral['io_sector_name'].fillna(iip_sectoral['sector_name'])

# Summary statistics
mapped_count = iip_sectoral['sector_name'].map(sector_mapping).notna().sum()
total_iip_sectors = iip_sectoral['sector_name'].nunique()

print(f"✓ Mapped {len(sector_mapping)} IIP manufacturing sectors to I-O sectors")
print(f"✓ Successfully mapped: {mapped_count:,} rows")
print(f"✓ IIP sectors: {total_iip_sectors}")
print(f"✓ Unmapped sectors: {iip_sectoral[iip_sectoral['io_sector_name'] == iip_sectoral['sector_name']]['sector_name'].unique()}")

✓ Mapped 22 IIP manufacturing sectors to I-O sectors
✓ Successfully mapped: 3,366 rows
✓ IIP sectors: 23
✓ Unmapped sectors: ['General Index']


In [13]:
# Display sample mappings
mapping_display = pd.DataFrame({
    'IIP Sector': list(sector_mapping.keys())[:10],
    'I-O Sector': list(sector_mapping.values())[:10]
})
display(mapping_display)

print(f"\n✓ Sector mapping complete!")

,IIP Sector,I-O Sector
0,Manufacture of food products,Miscellaneous food\n products
1,Manufacture of beverages,Beverages
2,Manufacture of textiles,Cotton textiles
3,Manufacture of wearing apparel,Ready made garments
4,Manufacture of leather and related products,Leather and leather\n products
5,Manufacture of paper and paper products,"Paper, Paper products and\n newsprint"
6,Printing and reproduction of recorded media,"Publishing, printing and\n allied activities"
7,Manufacture of chemicals and chemical products,Other chemicals
8,"Manufacture of pharmaceuticals, medicinal chem...",Drugs and medicine
9,Manufacture of coke and refined petroleum prod...,Petroleum products



✓ Sector mapping complete!


# Merge IIP with Network Metrics

In [14]:
# Start with IIP as base (time-series × sectors)
master_df = iip_sectoral.merge(
    network_metrics,
    left_on='io_sector_name',
    right_on='sector_name',
    how='left',
    suffixes=('', '_io')
)

In [15]:
# Drop duplicate sector_name column
master_df = master_df.drop(columns=['sector_name_io'], errors='ignore')

print(f"✓ Master dataset shape: {master_df.shape}")
print(f"✓ Date range: {master_df['date'].min()} to {master_df['date'].max()}")
print(f"✓ Unique sectors: {master_df['io_sector_name'].nunique()}")
print(f"✓ Unique months: {master_df['date'].nunique()}")

print("\nSample merged data:")
display(master_df[['date', 'sector_name', 'io_sector_name', 'iip_index', 'pagerank', 'backward_linkage']].head(5))

✓ Master dataset shape: (3519, 16)
✓ Date range: 2012-04-01 00:00:00 to 2024-12-01 00:00:00
✓ Unique sectors: 23
✓ Unique months: 153

Sample merged data:


,date,sector_name,io_sector_name,iip_index,pagerank,backward_linkage
0,2012-04-01,General Index,General Index,99.3,NaN,NaN
1,2012-05-01,General Index,General Index,105.0,NaN,NaN
2,2012-06-01,General Index,General Index,102.3,NaN,NaN
3,2012-07-01,General Index,General Index,101.5,NaN,NaN
4,2012-08-01,General Index,General Index,99.9,NaN,NaN


In [16]:
# Drop duplicate sector_name column
master_df = master_df.drop(columns=['sector_name_io'], errors='ignore')

print(f"✓ Master dataset shape: {master_df.shape}")
print(f"✓ Date range: {master_df['date'].min()} to {master_df['date'].max()}")
print(f"✓ Unique sectors: {master_df['io_sector_name'].nunique()}")
print(f"✓ Unique months: {master_df['date'].nunique()}")

print("\nSample merged data:")
display(master_df[['date', 'sector_name', 'io_sector_name', 'iip_index', 'pagerank', 'backward_linkage']].head(5))

✓ Master dataset shape: (3519, 16)
✓ Date range: 2012-04-01 00:00:00 to 2024-12-01 00:00:00
✓ Unique sectors: 23
✓ Unique months: 153

Sample merged data:


,date,sector_name,io_sector_name,iip_index,pagerank,backward_linkage
0,2012-04-01,General Index,General Index,99.3,NaN,NaN
1,2012-05-01,General Index,General Index,105.0,NaN,NaN
2,2012-06-01,General Index,General Index,102.3,NaN,NaN
3,2012-07-01,General Index,General Index,101.5,NaN,NaN
4,2012-08-01,General Index,General Index,99.9,NaN,NaN


# Merge Commodity Prices

In [17]:
master_df = master_df.merge(commodity_prices, on='date', how='left')
print(f"Shape after commodity prices: {master_df.shape}")

Shape after commodity prices: (3519, 47)


In [18]:
# Show which commodity columns were added
commodity_cols = [col for col in commodity_prices.columns if col !="date"]
print(f"\n✓ Added {len(commodity_cols)} commodity-related columns:")
print(f"  - Price columns: {len([c for c in commodity_cols if 'price' in c.lower() or c.isupper()])}")
print(f"  - Return columns: {len([c for c in commodity_cols if 'logret' in c])}")
print(f"  - Volatility columns: {len([c for c in commodity_cols if 'vol' in c])}")
print(f"  - Shock columns: {len([c for c in commodity_cols if 'shock' in c])}")


✓ Added 31 commodity-related columns:
  - Price columns: 5
  - Return columns: 5
  - Volatility columns: 15
  - Shock columns: 5


In [19]:
display(master_df.head())

,sector_name,date,iip_index,iip_mom_growth,iip_yoy_growth,is_energy_intensive,io_sector_name,sector_id,backward_linkage,forward_linkage,...,COPPER_vol_6m,COPPER_vol_12m,ALUMINUM_vol_3m,ALUMINUM_vol_6m,ALUMINUM_vol_12m,CRUDE_PETRO_shock,WHEAT_US_HRW_shock,RICE_05_shock,COPPER_shock,ALUMINUM_shock
0,General Index,2012-04-01,99.3,NaN,NaN,False,General Index,NaN,NaN,NaN,...,0.031643,0.058625,0.046561,0.046538,0.036056,0,0,0,0,0
1,General Index,2012-05-01,105.0,5.740181,NaN,False,General Index,NaN,NaN,NaN,...,0.039656,0.057612,0.028027,0.043486,0.035953,0,0,1,0,0
2,General Index,2012-06-01,102.3,-2.571429,NaN,False,General Index,NaN,NaN,NaN,...,0.050967,0.059555,0.023840,0.048468,0.037558,0,0,0,0,0
3,General Index,2012-07-01,101.5,-0.782014,NaN,False,General Index,NaN,NaN,NaN,...,0.042973,0.055663,0.027507,0.035105,0.037750,0,1,0,0,0
4,General Index,2012-08-01,99.9,-1.576355,NaN,False,General Index,NaN,NaN,NaN,...,0.032454,0.053458,0.028257,0.025249,0.036151,0,0,0,0,0


# Merge Climate, WPI, GDP, Global Macro

In [20]:
# ONI Climate
master_df = master_df.merge(climate_oni, on="date", how="left")

# WPI-Inflaiton - Pivot to wide format: 
wpi_wide= wpi_inflation.pivot(
    index='date', 
    columns='category', 
    values='wpi_inflation'
).reset_index()
wpi_wide.columns = ['date'] + [
    f'wpi_{col.lower().replace(" ", "_").replace("&", "and").replace(".", "")}'
    for col in wpi_wide.columns[1:]
]
master_df = master_df.merge(wpi_wide, on='date', how='left')

# GDP Quarterly 
master_df = master_df.merge(gdp_quarterly, on="date", how="left")

# Global Macro (OECD)
master_df = master_df.merge(global_macro, on='date', how='left')

In [21]:
print("\nSample with all time-series data:")
cols_to_show = ['date', 'sector_name', 'iip_index', 'CRUDE_PETRO', 'ONI', 'gdp_growth_yoy', 'g20_avg_cpi_growth']
cols_available = [c for c in cols_to_show if c in master_df.columns]
display(master_df[cols_available].head(3))
display(master_df.head(7))


Sample with all time-series data:


,date,sector_name,iip_index,CRUDE_PETRO,ONI,gdp_growth_yoy,g20_avg_cpi_growth
0,2012-04-01,General Index,99.3,113.665500,-0.3,NaN,NaN
1,2012-05-01,General Index,105.0,104.086034,-0.1,NaN,NaN
2,2012-06-01,General Index,102.3,90.728254,0.1,NaN,NaN


,sector_name,date,iip_index,iip_mom_growth,iip_yoy_growth,is_energy_intensive,io_sector_name,sector_id,backward_linkage,forward_linkage,...,GBR,IDN,IND,ITA,KOR,SAU,TUR,USA,ZAF,g20_avg_cpi_growth
0,General Index,2012-04-01,99.3,NaN,NaN,False,General Index,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,General Index,2012-05-01,105.0,5.740181,NaN,False,General Index,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,General Index,2012-06-01,102.3,-2.571429,NaN,False,General Index,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,General Index,2012-07-01,101.5,-0.782014,NaN,False,General Index,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,General Index,2012-08-01,99.9,-1.576355,NaN,False,General Index,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,General Index,2012-09-01,99.0,-0.900901,NaN,False,General Index,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,General Index,2012-10-01,103.7,4.747475,NaN,False,General Index,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Apply Fix - Remove General Index

# Remove "General Index" (aggregate index, not a real sector)
before_count = len(master_df)
master_df = master_df[master_df['sector_name'] != 'General Index'].copy()
after_count = len(master_df)

print(f"\n✓ Removed 'General Index':")
print(f"  Before: {before_count:,} rows")
print(f"  After:  {after_count:,} rows")
print(f"  Removed: {before_count - after_count:,} rows")

# Verify all rows now have network metrics
missing_network = master_df['sector_id'].isna().sum()
print(f"\n✓ Network metrics coverage: {len(master_df) - missing_network:,} / {len(master_df):,} ({(1 - missing_network/len(master_df))*100:.1f}%)")

# Verify date ranges
print(f"\n✓ Final date range: {master_df['date'].min()} to {master_df['date'].max()}")
print(f"✓ Unique manufacturing sectors: {master_df['sector_name'].nunique()}")

# Check for remaining issues
print("\n" + "="*70)
print("QUALITY CHECK")
print("="*70)

key_vars = ['iip_index', 'sector_id', 'pagerank', 'backward_linkage', 'CRUDE_PETRO']
for var in key_vars:
    if var in master_df.columns:
        missing = master_df[var].isna().sum()
        print(f"  {var:20} {missing:,} missing ({missing/len(master_df)*100:.1f}%)")

print("\n✓ Cleaned dataset ready!")
print("\nSample:")
display(master_df[['date', 'sector_name', 'iip_index', 'sector_id', 'pagerank', 'CRUDE_PETRO']].head())


✓ Removed 'General Index':
  Before: 3,634 rows
  After:  3,476 rows
  Removed: 158 rows

✓ Network metrics coverage: 3,476 / 3,476 (100.0%)

✓ Final date range: 2012-04-01 00:00:00 to 2024-12-01 00:00:00
✓ Unique manufacturing sectors: 22

QUALITY CHECK
  iip_index            0 missing (0.0%)
  sector_id            0 missing (0.0%)
  pagerank             0 missing (0.0%)
  backward_linkage     0 missing (0.0%)
  CRUDE_PETRO          0 missing (0.0%)

✓ Cleaned dataset ready!

Sample:


,date,sector_name,iip_index,sector_id,pagerank,CRUDE_PETRO
158,2012-04-01,Manufacture of basic metals,104.1,80.0,0.009427,113.665500
159,2012-05-01,Manufacture of basic metals,119.3,80.0,0.009427,104.086034
160,2012-06-01,Manufacture of basic metals,106.4,80.0,0.009427,90.728254
161,2012-07-01,Manufacture of basic metals,109.5,80.0,0.009427,96.754113
162,2012-08-01,Manufacture of basic metals,101.3,80.0,0.009427,105.273637


# Trade Exposure variables

In [23]:
# 8.1 Total trade by date
trade_totals = trade_bilateral.groupby('date').agg({
    'trade_value_usd': 'sum'
}).reset_index()
trade_totals.columns = ['date', 'total_trade_value']
print(f"✓ Calculated total trade for {len(trade_totals)} months")

✓ Calculated total trade for 180 months


In [24]:
# 8.2 Energy trade (if commodity_group exists)
if 'commodity_group' in trade_bilateral.columns:
    energy_trade = trade_bilateral[
        trade_bilateral['commodity_group'] == 'Energy'
    ].groupby('date')['trade_value_usd'].sum().reset_index()
    energy_trade.columns = ['date', 'energy_trade_value']
    
    master_df = master_df.merge(energy_trade, on='date', how='left')
    print(f"✓ Added energy trade exposure")

✓ Added energy trade exposure


In [25]:
# 8.3 Merge total trade
master_df = master_df.merge(trade_totals, on='date', how='left')
print(f"✓ Added total trade value")

print(f"\n✓ Shape after trade exposure: {master_df.shape}")

✓ Added total trade value

✓ Shape after trade exposure: (3476, 82)


In [26]:
# Show trade columns
trade_cols = [c for c in master_df.columns if 'trade' in c.lower()]
print(f"✓ Trade-related columns: {trade_cols}")

✓ Trade-related columns: ['energy_trade_value', 'total_trade_value']


In [27]:
master_df.head()

,sector_name,date,iip_index,iip_mom_growth,iip_yoy_growth,is_energy_intensive,io_sector_name,sector_id,backward_linkage,forward_linkage,...,IND,ITA,KOR,SAU,TUR,USA,ZAF,g20_avg_cpi_growth,energy_trade_value,total_trade_value
0,Manufacture of basic metals,2012-04-01,104.1,NaN,NaN,True,Iron and steel foundries,80.0,2.590097,3.839517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Manufacture of basic metals,2012-05-01,119.3,14.601345,NaN,True,Iron and steel foundries,80.0,2.590097,3.839517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Manufacture of basic metals,2012-06-01,106.4,-10.813076,NaN,True,Iron and steel foundries,80.0,2.590097,3.839517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Manufacture of basic metals,2012-07-01,109.5,2.913534,NaN,True,Iron and steel foundries,80.0,2.590097,3.839517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Manufacture of basic metals,2012-08-01,101.3,-7.488584,NaN,True,Iron and steel foundries,80.0,2.590097,3.839517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create Derived Variables

In [28]:
# 9.1 Energy intensity flag
energy_intensive_sectors = [
    'Petroleum products', 'Other chemicals', 'Iron and steel foundries',
    'Cement', 'Fertilizers', 'Inorganic heavy chemicals', 'Organic heavy chemicals',
    'Coal tar products', 'Synthetic fibers, resin'
]

master_df['is_energy_intensive'] = master_df['io_sector_name'].isin(energy_intensive_sectors)
energy_count = master_df['is_energy_intensive'].sum()
print(f"✓ Energy-intensive observations: {energy_count:,} ({energy_count/len(master_df)*100:.1f}%)")

✓ Energy-intensive observations: 632 (18.2%)


In [29]:
# 9.2 Interaction terms (if columns exist)
if 'pagerank' in master_df.columns and 'CRUDE_PETRO_shock' in master_df.columns:
    master_df['oil_shock_x_pagerank'] = master_df['CRUDE_PETRO_shock'] * master_df['pagerank']
    print("✓ Created oil_shock_x_pagerank interaction")

if 'betweenness_centrality' in master_df.columns and 'CRUDE_PETRO_shock' in master_df.columns:
    master_df['oil_shock_x_betweenness'] = master_df['CRUDE_PETRO_shock'] * master_df['betweenness_centrality']
    print("✓ Created oil_shock_x_betweenness interaction")

✓ Created oil_shock_x_pagerank interaction
✓ Created oil_shock_x_betweenness interaction


In [30]:
# 9.3 Lagged variables
print("\n  Creating lagged variables...")
master_df = master_df.sort_values(['io_sector_name', 'date'])

# Lag commodity prices
commodity_price_cols = ['CRUDE_PETRO', 'WHEAT_US_HRW', 'RICE_05', 'COPPER', 'ALUMINUM']
commodity_price_cols = [c for c in commodity_price_cols if c in master_df.columns]

for col in commodity_price_cols:
    master_df[f'{col}_lag1'] = master_df.groupby('io_sector_name')[col].shift(1)

print(f"✓ Lagged {len(commodity_price_cols)} commodity prices (1 month)")

# Lag IIP growth
if 'iip_yoy_growth' in master_df.columns:
    master_df['iip_yoy_growth_lag1'] = master_df.groupby('io_sector_name')['iip_yoy_growth'].shift(1)
    print("✓ Lagged IIP year-over-year growth")


  Creating lagged variables...
✓ Lagged 5 commodity prices (1 month)
✓ Lagged IIP year-over-year growth


In [31]:
# 9.4 Time indicators
master_df['year'] = master_df['date'].dt.year
master_df['month'] = master_df['date'].dt.month
master_df['quarter'] = master_df['date'].dt.quarter

print("✓ Added year, month, quarter indicators")

print(f"\n✓ Final shape after derived variables: {master_df.shape}")

✓ Added year, month, quarter indicators

✓ Final shape after derived variables: (3476, 93)


# Data Quality Checks

In [32]:
print(f"\nFinal dataset shape: {master_df.shape[0]:,} rows × {master_df.shape[1]} columns")
print(f"Date range: {master_df['date'].min()} to {master_df['date'].max()}")
print(f"Unique sectors: {master_df['io_sector_name'].nunique()}")
print(f"Unique months: {master_df['date'].nunique()}")
print(f"Memory usage: {master_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")


Final dataset shape: 3,476 rows × 93 columns
Date range: 2012-04-01 00:00:00 to 2024-12-01 00:00:00
Unique sectors: 22
Unique months: 153
Memory usage: 3.2 MB


In [33]:
# Missing Values in Key-Columns
key_cols = ['iip_index', 'iip_yoy_growth', 'CRUDE_PETRO', 'pagerank', 
            'backward_linkage', 'gdp_growth_yoy', 'g20_avg_cpi_growth']

missing_data = []
for col in key_cols:
    if col in master_df.columns:
        missing_count = master_df[col].isna().sum()
        missing_pct = (missing_count / len(master_df)) * 100
        missing_data.append({
            'Column': col,
            'Missing': f"{missing_count:,}",
            'Percentage': f"{missing_pct:.2f}%"
        })

missing_df = pd.DataFrame(missing_data)
display(missing_df)

,Column,Missing,Percentage
0,iip_index,0,0.00%
1,iip_yoy_growth,286,8.23%
2,CRUDE_PETRO,0,0.00%
3,pagerank,0,0.00%
4,backward_linkage,0,0.00%
5,gdp_growth_yoy,"1,034",29.75%
6,g20_avg_cpi_growth,"3,388",97.47%


In [34]:
col_categories = {
    'Identifiers': len([c for c in master_df.columns if any(x in c.lower() for x in ['sector', 'date', 'year', 'month', 'quarter'])]),
    'IIP variables': len([c for c in master_df.columns if 'iip' in c.lower()]),
    'Commodity prices': len([c for c in master_df.columns if any(x in c for x in ['CRUDE', 'WHEAT', 'RICE', 'COPPER', 'ALUMINUM'])]),
    'Network metrics': len([c for c in master_df.columns if any(x in c.lower() for x in ['linkage', 'centrality', 'pagerank'])]),
    'Macro variables': len([c for c in master_df.columns if any(x in c.lower() for x in ['gdp', 'wpi', 'oni', 'g20'])]),
    'Trade variables': len([c for c in master_df.columns if 'trade' in c.lower()]),
    'Derived variables': len([c for c in master_df.columns if any(x in c.lower() for x in ['lag', 'shock', 'intensive', 'interaction'])])
}

for category, count in col_categories.items():
    print(f"  {category:20} {count:3} columns")

print(f"\n  {'TOTAL':20} {master_df.shape[1]:3} columns")

  Identifiers           11 columns
  IIP variables          4 columns
  Commodity prices      35 columns
  Network metrics        8 columns
  Macro variables       11 columns
  Trade variables        2 columns
  Derived variables     17 columns

  TOTAL                 93 columns


### Save the Master Dataset

In [36]:
import os

In [37]:
# 11.1 Save full dataset
output_path = rootPath + 'master_dataset.csv'
master_df.to_csv(output_path, index=False)
print(f"\n✓ Saved full dataset: {output_path}")
print(f"  {master_df.shape[0]:,} rows × {master_df.shape[1]} columns")
print(f"  File size: {os.path.getsize(output_path) / 1024**2:.1f} MB")


✓ Saved full dataset: ./processed/master_dataset.csv
  3,476 rows × 93 columns
  File size: 3.1 MB


In [38]:
# 11.2 Save filtered dataset (2010-2024, non-missing IIP)
master_filtered = master_df[
    (master_df['date'] >= '2010-01-01') &
    (master_df['date'] <= '2024-12-31') &
    (master_df['iip_index'].notna())
].copy()

filtered_path = rootPath + 'master_dataset_filtered.csv'
master_filtered.to_csv(filtered_path, index=False)
print(f"\n✓ Saved filtered dataset: {filtered_path}")
print(f"  {master_filtered.shape[0]:,} rows × {master_filtered.shape[1]} columns")
print(f"  File size: {os.path.getsize(filtered_path) / 1024**2:.1f} MB")
print(f"  Removed {len(master_df) - len(master_filtered):,} rows with missing/out-of-range data")


✓ Saved filtered dataset: ./processed/master_dataset_filtered.csv
  3,476 rows × 93 columns
  File size: 3.1 MB
  Removed 0 rows with missing/out-of-range data


In [39]:
# 11.3 Save column metadata
columns_df = pd.DataFrame({
    'column_name': master_df.columns,
    'dtype': master_df.dtypes.astype(str),
    'non_null_count': master_df.count(),
    'null_count': master_df.isna().sum(),
    'null_percentage': (master_df.isna().sum() / len(master_df) * 100).round(2)
})

columns_path = rootPath + 'master_dataset_columns.csv'
columns_df.to_csv(columns_path, index=False)
print(f"\n✓ Saved column metadata: {columns_path}")


✓ Saved column metadata: ./processed/master_dataset_columns.csv
